<h5> Description </h5>
<ol>
 <li> Identify the features we want to use
</ol>

In [2]:
#uncomment for jupyter notebook
#import findspark
#findspark.init()

#import pyspark
#sc = pyspark.SparkContext()
#spark = pyspark.sql.SparkSession(sc)

# General imports
from datetime import datetime
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler

s = SQLContext(sc)

<h5> Convert user logs into monthly data </h5>

In [4]:
def renameCols(df,newNames):
    nDF = df
    assert(len(df.schema.names) == len(newNames))
    for i in range(0,len(df.schema.names)):
        nDF = nDF.withColumnRenamed(nDF.schema.names[i],newNames[i])
    return nDF

In [5]:
d = s.sql('select * from unmgmt_user_logs_all_table')
r = d.rdd.map(lambda l: (l[0],l[1],l[2],l[3],l[4],l[5],l[6],l[7],l[8],l[1].year,l[1].month,  (2017-l[1].year)*12 - l[1].month + 1) )

In [6]:
usrLogWithYM = renameCols(r.toDF(), d.schema.names + ['year','month','months_b4_Feb2017'])
usrLogWithYM.show(1)

In [7]:
usrGroup = usrLogWithYM.groupby(['msno','year','month']).agg({ 'num_25': 'sum' , 'num_50': 'sum' ,'num_75': 'sum' ,'num_985': 'sum' ,'num_100': 'sum' ,'num_unq': 'sum','total_secs': 'sum','months_b4_Feb2017' : 'avg'})

In [8]:
colNames = ','.join(usrGroup.schema.names).replace('(','_').replace(')','').split(',')

In [9]:
renamed = renameCols(usrGroup, colNames)

In [10]:
renamed.write.saveAsTable('usrGroupByMonthAndYear')

<h5> Assemble the data we want to train </h5>

In [12]:
assembler = VectorAssembler ( inputCols = ['sum(num_25)','sum(num_50)','sum(num_75)','sum(num_985)','sum(num_100)','sum(num_unq)','sum(total_secs)','avg(months_b4_Feb2017)'], outputCol = 'features')

In [13]:
trainData = assembler.transform(usrGroup).select(['features'])

In [14]:
trainData.show(1)